# **Розширений план-контент для групового заняття**  
## **Тема 2. Заняття 9. Вбудовані процедури та функції. Поняття про тригери та транзакції.**  
### **1. Вбудовані процедури та функції.**  
### **2. Поняття про тригери та транзакції.**  

---

## **1. Вступ (10 хв)**  

### **1.1. Мета заняття**  
- Ознайомити слухачів із вбудованими процедурами та функціями у PostgreSQL.  
- Навчити використовувати механізм тригерів для автоматизації операцій у базі даних.  
- Розглянути основи транзакцій та управління ними в PostgreSQL.  
- Виконати практичні завдання для засвоєння матеріалу.  

### **1.2. Очікувані результати**  
Після заняття слухачі зможуть:  
✅ Використовувати вбудовані функції PostgreSQL (`STRING`, `DATE`, `AGGREGATE`).  
✅ Створювати власні **збережені процедури** та **функції**.  
✅ Використовувати **тригери** для автоматизації змін у базі.  
✅ Керувати **транзакціями** (`COMMIT`, `ROLLBACK`, `SAVEPOINT`).  

---

## **2. Вбудовані процедури та функції (40 хв)**  

### **2.1. Огляд вбудованих функцій PostgreSQL**  
📌 **Типи вбудованих функцій:**
1. **Функції роботи з рядками (`STRING FUNCTIONS`)**  
2. **Функції роботи з датами (`DATE FUNCTIONS`)**  
3. **Агрегатні функції (`AGGREGATE FUNCTIONS`)**  

---

### **2.2. Функції роботи з рядками**  
📌 **Приклади роботи з рядками:**  
```sql
SELECT UPPER('капітан'); -- ВЕРХНІЙ РЕГІСТР
SELECT LOWER('ЛЕЙТЕНАНТ'); -- НИЖНІЙ РЕГІСТР
SELECT CONCAT('Командир', ' ', 'Батальйону'); -- Об'єднання рядків
SELECT LENGTH('Військовий'); -- Довжина рядка
```

---

### **2.3. Функції роботи з датами**  
📌 **Приклади обчислення дат:**  
```sql
SELECT CURRENT_DATE; -- Поточна дата
SELECT AGE('2000-01-01'); -- Вік у роках від заданої дати
SELECT NOW(); -- Поточний час
SELECT DATE_PART('year', NOW()); -- Витягнення року з дати
```

---

### **2.4. Агрегатні функції**  
📌 **Приклади використання агрегатних функцій:**  
```sql
SELECT COUNT(*) FROM Personnel; -- Кількість військовослужбовців
SELECT AVG(salary) FROM Personnel; -- Середня зарплата
SELECT MAX(salary) FROM Personnel; -- Максимальна зарплата
SELECT MIN(salary) FROM Personnel; -- Мінімальна зарплата
```

---

### **2.5. Створення власних процедур та функцій**  
📌 **Функція, що повертає звання військовослужбовця за ID:**  
```sql
CREATE OR REPLACE FUNCTION get_rank(p_id INT) RETURNS VARCHAR AS $$
DECLARE
    v_rank VARCHAR;
BEGIN
    SELECT rank INTO v_rank FROM Personnel WHERE personnel_id = p_id;
    RETURN v_rank;
END;
$$ LANGUAGE plpgsql;
```
📌 **Виклик функції:**  
```sql
SELECT get_rank(1);
```

📌 **Процедура для оновлення звання військового:**  
```sql
CREATE OR REPLACE PROCEDURE update_rank(p_id INT, new_rank VARCHAR) AS $$
BEGIN
    UPDATE Personnel SET rank = new_rank WHERE personnel_id = p_id;
END;
$$ LANGUAGE plpgsql;
```
📌 **Виклик процедури:**  
```sql
CALL update_rank(1, 'Полковник');
```

---

## **3. Поняття про тригери та транзакції (50 хв)**  

### **3.1. Що таке тригери?**  
📌 **Тригери** – це механізм у базі даних, який автоматично виконує певну операцію при зміні даних у таблиці.  

**Основні види тригерів:**
- **BEFORE** – виконується до внесення змін.  
- **AFTER** – виконується після внесення змін.  
- **INSTEAD OF** – замінює операцію.  

---

### **3.2. Приклад створення тригера**
📌 **Автоматичне оновлення дати зміни запису**  
```sql
ALTER TABLE Personnel ADD COLUMN updated_at TIMESTAMP DEFAULT NOW();

CREATE OR REPLACE FUNCTION update_timestamp() RETURNS TRIGGER AS $$
BEGIN
    NEW.updated_at = NOW();
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER personnel_update_trigger
BEFORE UPDATE ON Personnel
FOR EACH ROW
EXECUTE FUNCTION update_timestamp();
```
📌 **Тестування тригера:**  
```sql
UPDATE Personnel SET rank = 'Майор' WHERE personnel_id = 1;
SELECT * FROM Personnel;
```

---

### **3.3. Основи транзакцій**
📌 **Транзакція** – це група SQL-операцій, що виконується як єдине ціле.  
Використовуються команди:
- **`BEGIN`** – початок транзакції.  
- **`COMMIT`** – збереження змін.  
- **`ROLLBACK`** – скасування змін.  
- **`SAVEPOINT`** – точка відкату у транзакції.  

📌 **Приклад використання транзакцій:**  
```sql
BEGIN;
UPDATE Personnel SET rank = 'Генерал' WHERE personnel_id = 1;
ROLLBACK; -- Відміна змін
```
📌 **Використання `SAVEPOINT`**  
```sql
BEGIN;
UPDATE Personnel SET rank = 'Генерал' WHERE personnel_id = 1;
SAVEPOINT sp1;
UPDATE Personnel SET rank = 'Маршал' WHERE personnel_id = 2;
ROLLBACK TO sp1; -- Відкат до SAVEPOINT
COMMIT;
```

---

## **4. Практичні завдання (30 хв)**  
✅ **Завдання 1:** **Створити функцію, яка повертає назву підрозділу за ID військового.**  
✅ **Завдання 2:** **Створити процедуру, яка змінює місце дислокації підрозділу.**  
✅ **Завдання 3:** **Створити тригер, що автоматично записує зміни у підрозділах.**  
✅ **Завдання 4:** **Виконати транзакцію з кількома оновленнями та точкою збереження.**  

📌 **Приклад рішення завдання 1:**  
```sql
CREATE OR REPLACE FUNCTION get_unit(p_id INT) RETURNS VARCHAR AS $$
DECLARE
    v_unit VARCHAR;
BEGIN
    SELECT unit INTO v_unit FROM Personnel WHERE personnel_id = p_id;
    RETURN v_unit;
END;
$$ LANGUAGE plpgsql;
```
📌 **Приклад рішення завдання 4:**  
```sql
BEGIN;
UPDATE Personnel SET rank = 'Генерал' WHERE personnel_id = 1;
SAVEPOINT sp1;
UPDATE Personnel SET unit = 'Батальйон C' WHERE personnel_id = 2;
ROLLBACK TO sp1;
COMMIT;
```

---

## **5. Висновки та підсумки заняття (10 хв)**  
✅ **Основні висновки**:
- **Функції та процедури** дозволяють автоматизувати запити.
- **Тригери** використовуються для автоматичних оновлень записів.
- **Транзакції** допомагають контролювати зміни в базі даних.

🚀 **Ключовий меседж**:  
**Використання процедур, тригерів та транзакцій підвищує безпеку та ефективність роботи з базою даних, що критично важливо у військових аналітичних системах.**